In [1]:
import re
import pandas as pd 
# Các đoạn mô tả


In [12]:


def split_sentences_on_plus_minus(text):
    regex = r'\s+([+-])\s+'
    sentences = re.split(regex, text)
    output = []
    for i in range(len(sentences)):
        if sentences[i].strip() in ['+', '-']:
            output.append(sentences[i] + sentences[i+1])
        elif sentences[i-1].strip() not in ['+', '-']and i > 0:
            output.append(sentences[i])
    return output

def extract_sentences(text):
    # Biểu thức chính quy để tìm câu chứa từ 'fee', '%' hoặc 'VND'
    regex = r'[^.!?]*?(mile|km)[^.!?]*?[.!?]'
    
    # Tìm các câu phù hợp với biểu thức chính quy
    matches = re.finditer(regex, text)
    
    # Tạo danh sách các câu phù hợp
    sentences = [match.group(0).strip() for match in matches]
    
    return sentences

def process_descriptions(df):
    # Duyệt qua từng hàng trong cột 'describe'
    for index, row in df.iterrows():
        sentences = extract_sentences(row['describe'])  # Trích xuất các câu từ mô tả
        processed_sentences = []  # Danh sách các câu đã được xử lý
        # Duyệt qua các câu đã trích xuất
        for sentence in sentences:
            sentence_x = split_sentences_on_plus_minus(sentence)
            processed_sentences.extend(sentence_x)  # Thêm câu đã xử lý vào danh sách
        # Ghép các câu đã xử lý lại thành một đoạn văn và ghi vào DataFrame
        processed_text = " ".join(processed_sentences)
        df.at[index, 'describe'] = processed_text

def count_empty_rows_in_describe_column(df):
    # Đếm số hàng có giá trị rỗng (dấu trống hoặc NaN)
    empty_rows_count = df['describe'].apply(lambda x: x.strip() != '').sum()
    return empty_rows_count



In [13]:
df = pd.read_csv('car_detail_en.csv')
process_descriptions(df)
empty_rows_count = count_empty_rows_in_describe_column(df)
print(empty_rows_count)
print(df['describe'].head(20))
df.shape

1646
0     
1     
2     
3     
4     
5     
6     
7     
8     
9     
10    
11    
12    
13    
14    
15    
16    
17    
18    
19    
Name: describe, dtype: object


(30652, 21)

In [14]:
result = df[df['describe'].apply(lambda x: x.strip() != '')]
print(result['describe'])

52       -5 years/100,00km manufacturer warranty Dimens...
58       -Cash discount and free high-end accessories p...
86       -3 years or 200,000 km, whichever comes first ...
112      +Chance to win a TV and a Vespa displayed at t...
123      -80% Preferential interest rate, neat document...
                               ...                        
30564    -Vehicle equipped: genuine leather seats, scre...
30606    -Provincial license plate -Odo: 14,000km price...
30615    -Provincial license plate -Odo: 35,000km Price...
30626     -beautiful car with screen, full reverse camera.
30650    -full LED headlight cluster with 2-layer desig...
Name: describe, Length: 1646, dtype: object


In [15]:
result.shape

(1646, 21)

In [ ]:
result.to_csv('mileage.csv')